In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
spark

### Смотрим что там в файлах

In [ ]:
!hdfs dfs -ls /labs/slaba03/ | awk '{print $8}' | xargs -I {} bash -c 'echo {}; hdfs dfs -head {} | sed -n "1,5p"'

### Определяем разделитель

In [ ]:
!hdfs dfs -head /labs/slaba03/laba03_items.csv | xxd | sed -n '1,5p'

In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, TimestampType, ArrayType

In [5]:
df_item = spark.read \
        .format("csv") \
        .option("header", "true") \
        .option("delimiter", "\t") \
        .option("inferSchema", "true") \
        .load('/labs/slaba03/laba03_items.csv')
print('Count ' + str(df_item.count()))
df_item.show(2, True, True)
df_item.printSchema()

Count 635568
-RECORD 0-------------------------------------------
 item_id                     | 65667                
 channel_id                  | null                 
 datetime_availability_start | 1970-01-01 03:00:00  
 datetime_availability_stop  | 2018-01-01 03:00:00  
 datetime_show_start         | null                 
 datetime_show_stop          | null                 
 content_type                | 1                    
 title                       | на пробах только ... 
 year                        | 2013.0               
 genres                      | Эротика              
 region_id                   | null                 
-RECORD 1-------------------------------------------
 item_id                     | 65669                
 channel_id                  | null                 
 datetime_availability_start | 1970-01-01 03:00:00  
 datetime_availability_stop  | 2018-01-01 03:00:00  
 datetime_show_start         | null                 
 datetime_show_stop          | nu

In [6]:
df_item.rdd.getNumPartitions()

2

In [7]:
df_item = df_item.repartition(6)

In [8]:
df_train = spark.read \
        .format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema", "true") \
        .load('/labs/slaba03/laba03_train.csv')
print('Count ' + str(df_train.count()))
df_train.show(2, True, True)
df_train.printSchema()

Count 5032624
-RECORD 0---------
 user_id  | 1654  
 item_id  | 74107 
 purchase | 0     
-RECORD 1---------
 user_id  | 1654  
 item_id  | 89249 
 purchase | 0     
only showing top 2 rows

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- purchase: integer (nullable = true)



In [9]:
df_train.rdd.getNumPartitions()

2

In [10]:
df_train = df_train.repartition(6)

In [11]:
df_test = spark.read \
        .format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema", "true") \
        .load('/labs/slaba03/laba03_test.csv')
print('Count ' + str(df_test.count()))
df_train.show(2, True, True)
df_train.printSchema()

Count 2156840
-RECORD 0----------
 user_id  | 903040 
 item_id  | 94901  
 purchase | 0      
-RECORD 1----------
 user_id  | 938142 
 item_id  | 88705  
 purchase | 0      
only showing top 2 rows

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- purchase: integer (nullable = true)



In [12]:
df_test.rdd.getNumPartitions()

2

In [13]:
df_test = df_test.repartition(6)

## Платно/Бесплатно

### Платно

In [14]:
dfp = df_item.filter(df_item.content_type == 1)

Дубликатов item_id нет

In [ ]:
dfp.select(dfp.item_id).count(), dfp.select(dfp.item_id).distinct().count()

In [ ]:
pids = [row.item_id for row in dfp.select(dfp.item_id).collect()]

In [ ]:
len(pids)

In [ ]:
tids = [row.item_id for row in df_train.select(df_train.item_id).distinct().collect()]

In [ ]:
len(tids)

In [ ]:
teids = [row.item_id for row in df_test.select(df_test.item_id).distinct().collect()]

In [ ]:
len(teids)

In [ ]:
i = 0
for p1, t, te in zip(sorted(pids), sorted(tids), sorted(teids)):
    i = i + 1
    if p1 != t or t != te:
        print("Лажа")
        break
i

## Вывод из всего этого? Можно дропнуть в датасете все бесплатные итемы

In [14]:
df_item = df_item.filter(df_item.content_type == 1)

In [ ]:
df_item.count()

### Дропаем все итемы без жанров ?

In [ ]:
df_item[F.isnull(df_item.genres)].select(df_item.item_id, df_item.year, df_item.title).collect()

Слишком популярные сериалы без жанров, нужно проставить им вручную жанры, но сначала нужно обработать дубликаты

In [ ]:
df_item.select(df_item.title).count(), df_item.select(df_item.title).distinct().count()

In [ ]:
df_item\
    .filter(df_item.content_type == 1)\
    .groupBy(df_item.title, df_item.year)\
    .count()\
    .filter(F.col('count') > 1)\
    .orderBy('count', ascending=False)\
    .show(1000)

In [ ]:
df_item\
    .filter(df_item.content_type == 1)\
    .groupBy(df_item.title, df_item.year)\
    .agg(F.collect_list("item_id"))\
    .show()

Как обработать дубликаты? Новая колонка item_id в которой указывать на один избранный дубликат

In [15]:
duplicate_ids = df_item\
    .groupBy(df_item.title, df_item.year)\
    .agg(F.collect_list('item_id').alias('ids'))\
    .select('ids')\
    .filter(F.size(F.col('ids')) > 1)\
    .collect()
cor_ids = {}
for ids in [row.ids for row in duplicate_ids]:
    for iids in ids[1:]:
        cor_ids[iids] = ids[0]
cor_ids

{73127: 74553,
 81780: 88771,
 89028: 93990,
 95092: 87546,
 99871: 77371,
 2136: 80900,
 78010: 96254,
 92932: 82802,
 10959: 88986,
 10664: 66595,
 66187: 83484,
 74294: 2632,
 87900: 9941,
 93683: 100203,
 88648: 89253,
 87890: 5103,
 94041: 1599,
 73309: 74646,
 82784: 7607,
 82289: 10339,
 78269: 67828,
 77835: 90775,
 72699: 88559,
 8636: 74620,
 88547: 94651,
 82306: 3083,
 99849: 77934,
 101602: 77934,
 98101: 102640,
 8586: 99122,
 99329: 8679,
 78082: 8679,
 101615: 8679,
 103931: 97271,
 93748: 100029,
 10320: 66442,
 89993: 74342,
 89022: 79862,
 74322: 9866,
 72858: 86729,
 10079: 77409,
 9861: 94192,
 101471: 9130,
 100217: 83568,
 8252: 67007,
 72845: 99785,
 77825: 98704,
 7393: 94846,
 10128: 74443,
 79844: 67397,
 88989: 88773,
 85991: 86876,
 74605: 540,
 78976: 98737,
 66399: 83512,
 73310: 74379,
 74805: 74115,
 3755: 73739,
 88886: 85740,
 99962: 85740,
 10031: 74523,
 9855: 95051,
 74380: 3408,
 95015: 89014,
 80895: 8634,
 79412: 92393,
 92357: 4603,
 66585: 822

In [ ]:
df_item.filter(df_item.title.like('%old')).select(df_item.title).show(8)

In [16]:
_ids = [9055, 100248, 9721, 100247]

for _id in _ids:
    if _id in cor_ids:
        print('Лажа')
        break

# американская история ужасов old
cor_ids[9055] = 100248
# карточный домик old
cor_ids[9721] = 100247

 Проверить на равенство жанров коррелирующие id

# Очень неоптимизированный код

In [ ]:
not_equal_genres_cor_ids = [id_ for id_ in cor_ids if df_item[df_item.item_id == id_][[df_item.genres]].collect() != df_item[df_item.item_id == cor_ids[id_]][[df_item.genres]].collect()]
not_equal_genres_cor_ids

In [ ]:
len(not_equal_genres_cor_ids), len(cor_ids)

У двух третей дубликатов разные жанры, здесь можно потом будет что-нибудь придумать, чтобы увеличить AUC ROC, пока лень

### Заполняем пробелы в жанрах

Создадим отдельный датафрейм, в котором руками укажем жанры для итемов, потом left join новая колонка, после when(F.isnull(df.genres), df.new_col).otherwise(df.genres)

In [ ]:
genres = set()
for row in df_item.select(F.split(df_item.genres, ',').alias('words')).collect():
    if row.words != None:
        for word in row.words:
            genres.add(word)
genres

In [ ]:
missing_genres = [
 (6151, 'Триллер,Драма,Криминал,Детективы,Сериалы,Зарубежные'),   #2011.0, title='родина'),
 (9055, 'Ужасы,Фантастика,Триллер,Драма,Сериалы,Зарубежные'),   #2011.0, title='американская история ужасов old'),
 (9059, 'Боевик,Триллер,Драма,Криминал,Детектив,Сериалы,Зарубежные'),   #2013.0, title='банши'),
 (9062, 'Комедия,Сериалы,Зарубежные'),   #2012.0, title='вице-президент'),
 (9064, 'Детективы,Криминал,Триллер,Драма,Сериалы,Зарубежные'),   #2014.0, title='настоящий детектив'),
 (9180, 'Фантастика,Драма,Боевик,Мелодрама,Приключения,Сериалы,Зарубежные'),   #2011.0, title='игра престолов'),
 (9632, 'Комедия,Сериалы,Зарубежные'),   #2014.0, title='силиконовая долина'),
 (9667, 'Драма,Криминал,Сериалы,Зарубежные'),   #2013.0, title='рэй донован'),
 (9668, 'Драма,Исторический,Сериалы,Зарубежные'),   #2015.0, title='покажите мне героя'),
 (9699, 'Драма,Музыкальные,Сериалы,Зарубежные'),   #2016.0, title='винил'),
 (9717, 'Драма,Исторический,Сериалы,Зарубежные'),   #2014.0, title='больница никербокер'),
 (9720, 'Боевик,Драма,Приключения,Сериалы,Зарубежные'),   #2014.0, title='черные паруса'),
 (9721, 'Драма,Сериалы,Зарубежные'),   #2013.0, title='карточный домик old'),
 (9817, 'Фантастика,Драма,Детективы,Сериалы,Зарубежные'),   #2014.0, title='оставленные'),
 (9819, 'Мультфильм,Комедия,Сериалы,Зарубежные'),   #2016.0, title='звери'),
 (9821, 'Драма,Сериалы,Зарубежные'),   #2015.0, title='плоть и кости'),
 (9896, 'Драма,Сериалы,Зарубежные'),   #2016.0, title='девушка по вызову'),
 (9897, 'Драма,Мелодрама,Комедия,Сериалы,Зарубежные'),   #2015.0, title='вместе'),
 (9898, 'Драма,Музыкальные,Сериалы,Зарубежные'),   #2015.0, title='империя'),
 (9914, 'Драма,Криминал,Сериалы,Зарубежные'),   #1999.0, title='клан сопрано'),
 (10205, 'Триллер,Криминал,Драма,Сериалы,Наши'),  #2015.0, title='метод'),
 (10208, 'Триллер,Криминал,Детективы,Сериалы,Зарубежные'),  #2011.0, title='мост'),
 (94973, 'Эротика'),  #2016.0, title='бруклин ли. дневник нимфоманки'),
 (94974, 'Эротика'),  #2016.0, title='душевные страсти'),
 (94976, 'Эротика'),  #2016.0, title='лисы в курятнике'),
 (94977, 'Эротика'),  #2015.0, title='новичкам везет (часть 1)'),
 (94979, 'Эротика'),  #2015.0, title='знает кошка, чью сметану съела'),
 (94975, 'Эротика'),  #2015.0, title='гламурный хардкор 3'),
 (94978, 'Эротика'),  #2016.0, title='сладкая помада'),
 (94988, 'Эротика'),  #2015.0, title='новичкам везет (часть 2)'),
 (100247, 'Драма,Сериалы,Зарубежные'), #2013.0, title='карточный домик'),
 (100248, 'Ужасы,Фантастика,Триллер,Драма,Сериалы,Зарубежные'), #2011.0, title='американская история ужасов'),
 (103377, 'Мультфильм,Короткометражки,Зарубежные'), #None, title='big buck bunny 1080p')
]

In [ ]:
df_mis_genres = spark.sparkContext.parallelize(missing_genres).toDF()\
    .select(F.col('_1').alias('item_id'), F.col('_2').alias('mis_genres'))
df_mis_genres.printSchema()

### Итемы Шаг 1. Приклеиваем доп колонкой отсутствующие жанры

In [ ]:
df_1 = df_item.join(df_mis_genres, on='item_id', how='leftouter').coalesce(6)

In [ ]:
df_1.printSchema()

### Итемы Шаг 2. merge genres and mis_genres

In [ ]:
df_2 = df_1.withColumn('genres_nn', F.coalesce(df_1.genres, df_1.mis_genres))
df_2.printSchema()

In [ ]:
df_2.filter(df_2.item_id.isin([9062,65667,9064]))\
    .select('item_id', 'genres', 'mis_genres', 'genres_nn')\
    .show(3, True, True)

### Итемы Шаг 3. Сплитим жанры в массив

In [ ]:
df_3 = df_2.withColumn("genres_arr", F.split(df_2.genres_nn, ','))

### Итемы Шаг 4. Коллапсируем синонимы жанров

In [ ]:
replace_dict = {}

In [ ]:
replace_dict[' сказка'] = 'Сказки'
replace_dict['Комедии'] = 'Комедия'
replace_dict['Арт-хаус'] = 'Артхаус'
replace_dict['Анимация'] = 'Мультфильм'
replace_dict['Боевики'] = 'Боевик'
replace_dict['Военные'] = 'Военный'
replace_dict['Детские песни'] = 'Детские'
replace_dict['Для самых маленьких'] = 'Детские'
replace_dict['Для детей'] = 'Детские'
replace_dict['Документальные'] = 'Документальный'
replace_dict['Драма'] = 'Драмы'
replace_dict['Детективы'] = 'Детектив'
replace_dict['Западные мультфильмы'] = 'Мультфильм'
replace_dict['Исторические'] = 'Исторический'
replace_dict['Комедии'] = 'Комедия'
replace_dict['Короткометражки'] = 'Короткометражные'
replace_dict['Мелодрама'] = 'Мелодрамы'
replace_dict['Музыкальные'] = 'Музыкальный'
replace_dict['Мультсериалы'] = 'Мультфильм'
replace_dict['Мультфильмы'] = 'Мультфильм'
replace_dict['Мультфильмы в 3D'] = 'Мультфильм'
replace_dict['Приключение'] = 'Приключения'
replace_dict['Наши'] = 'Русские'
replace_dict['Русские мультфильмы'] = 'Мультфильм'
replace_dict['Семейные'] = 'Семейный'
replace_dict['Советские'] = 'Русские'
replace_dict['Советское кино'] = 'Русские'
replace_dict['Союзмультфильм'] = 'Мультфильм'
replace_dict['Спорт'] = 'Спортивные'
replace_dict['Триллер'] = 'Триллеры'
replace_dict['Фантастика'] = 'Фантастические'
replace_dict['Фильмы в 3D'] = 'Фильмы'
replace_dict['Фэнтези'] = 'Фантастические'
replace_dict['Юмористические'] = 'Комедия'

In [ ]:
replace_dict_br = spark.sparkContext.broadcast(replace_dict)

In [ ]:
df_4 = df_3.withColumn('genres_arr_colapsed', F.pandas_udf(lambda series: series.apply(lambda arr: [replace_dict_br.value[word] if word in replace_dict_br.value else word for word in arr]), ArrayType(StringType()), F.PandasUDFType.SCALAR)(F.col('genres_arr')))

### Итемы Шаг 5. Подсчитываем количество вхождений слов в массив 

In [ ]:
from pyspark.ml.feature import CountVectorizer

In [ ]:
count_vectorizer = CountVectorizer(inputCol="genres_arr_colapsed",
                                   outputCol="genre_vector",
                                   vocabSize=51, # Посчитал заранее
                                   binary=True # binary - потому что заменяли синонимы, вдруг там дублирование по ним есть
                                  )

In [ ]:
cv_model = count_vectorizer.fit(df_4)

In [ ]:
df_5 = cv_model.transform(df_4)

In [ ]:
sorted(cv_model.vocabulary)

### Хочу посчитать количество всех покупок по жанрам

In [ ]:
df_train.printSchema()
df_train.count(), df_train.filter(df_train.purchase == 1).count()

In [ ]:
df_train.filter(df_train.purchase == 1).select(df_train.user_id).distinct().count()

In [ ]:
df_train.select(df_train.user_id).distinct().count(), df_train.select(df_train.item_id).distinct().count()

In [ ]:
df_test.select(df_test.user_id).distinct().count(), df_test.select(df_test.item_id).distinct().count()

In [ ]:
df_test.count()

In [ ]:
print(1941 * 3704)
print(5032624 + 2156840)

Датасет - декартовое произведение юзеров и итемов

In [ ]:
# Количество итемов, которых кто-либо купил
df_train.filter(df_train.purchase == 1).select(df_train.item_id).distinct().count()

Подклеиваем инфу по жанрам к каждой покупке/не покупке

### Train Шаг 1. Вклеиваем инфу по итемам в train

In [ ]:
dft_1 = df_train.filter(df_train.purchase == 1)\
    .join(df_5.select('item_id', 'genre_vector'), on='item_id')\
    .coalesce(6)\
    .cache()

Мапа с информацией фильм, количество покупок

In [ ]:
df_item_count = dft_1\
    .rdd\
    .map(lambda row: (row.item_id, 1))\
    .reduceByKey(lambda x, y: x + y)\
    .toDF()\
    .select(F.col('_1').alias('item_id'), F.col('_2').alias('cnt'))

In [ ]:
df_item_count.count()

### Итемы Шаг 6: Приклеиваем инфу по количеству покупок к итемам

In [ ]:
df_6 = df_5.join(df_item_count, on='item_id', how='leftouter').coalesce(6)
df_6.printSchema()

### Итемы Шаг 7: cnt null -> 0

In [ ]:
df_7 = df_6.withColumn('cunt', F.coalesce(df_6.cnt, F.lit(0))) # cunt вместо count, потому что на count начинает ругаться
df_7.printSchema()

### Итемы Шаг 8: genre_vector * cunt

In [ ]:
df_8 = df_7.withColumn('weight_vec',
                       F.udf(
                           lambda vec, cnt: (vec.toArray().astype(int) * cnt).tolist(),
                           ArrayType(IntegerType())
                       )('genre_vector','cunt'))

In [ ]:
df_8.printSchema()

Вычисляем общий вектор покупок по жанрам

In [ ]:
import numpy as np

In [ ]:
common_vec = df_8.rdd\
    .map(lambda row: (1, row.weight_vec))\
    .reduceByKey(lambda x, y: np.array(x) + np.array(y))\
    .take(1)[0][1]
common_vec

In [ ]:
for genre, count in sorted(zip(cv_model.vocabulary, common_vec), key=lambda row: row[1])[::-1]:
    print(genre + ': ' + str(count))

С итемами разобарлись, я так думаю, теперь надо разобраться с юзверами

### Получаем стату по юзерам (сколько юзер купил каких жанров) 

In [ ]:
dfu = dft_1.rdd.map(lambda row: (row.user_id, row.genre_vector.toArray().astype(int).tolist())).reduceByKey(lambda x, y: (np.array(x) + np.array(y)).tolist())\
    .toDF()\
    .select(F.col('_1').alias('user_id'), F.col('_2').alias('genre_cnt'))
dfu

In [ ]:
dfu.show(2, True, True)

### Сколько всего итемов купил каждый юзер

In [ ]:
dft_1.filter(dft_1.user_id == 754230).count()

In [ ]:
user_cnt = dft_1.select(dft_1.user_id, F.lit(1).alias('_1'))\
    .groupBy(dft_1.user_id)\
    .agg(F.count('_1').alias('cnt'))
dfu_1 = dfu.join(user_cnt, on='user_id')
dfu_1.count(), dfu.count()

In [ ]:
dfu_cnt0 = df_train.select(df_train.user_id)\
    .distinct()\
    .join(dfu_1.select(dfu_1.user_id), on='user_id', how="leftanti")\
    .coalesce(6)\
    .select('user_id', F.array([F.lit(0)] * len(cv_model.vocabulary)).alias('genre_cnt'), F.lit(0).alias('cnt'))

In [ ]:
dfu_cnt0.take(2)

###  Юзеры Шаг 2: объединяем платежников и халявщиков

In [ ]:
dfu_2 = dfu_1.union(dfu_cnt0)
dfu_2.printSchema()
dfu_2.count()

## Промежуточные итоги
dfu_2 - содержит инфу по юзерам, сколько каких жанров было куплено и общее количество купленных

df_8 - содержит инфу по итемам, сколько каких жанров было куплено и общее количество купленных

In [ ]:
dfri = F.broadcast(df_8.select(df_8.item_id, df_8.weight_vec.alias('ivec'), df_8.cunt.alias('icount')).cache())

In [ ]:
dfru = F.broadcast(dfu_2.select(dfu_2.user_id, dfu_2.genre_cnt.alias('uvec'), dfu_2.cnt.alias('ucount')).cache())

In [ ]:
dfri, dfru

## Попробуем намутить фичи

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import DenseVector

### Перемножим вектора юзеров и итемов по жанрам + общее количество юзеров и итемов

In [ ]:
dft = df_train.join(dfri, on='item_id')\
    .join(dfru, on='user_id')\
    .coalesce(6);

In [ ]:
dft

In [ ]:
from pyspark.ml.linalg import VectorUDT

In [ ]:
dft1 = dft.withColumn('mvec', F.udf(lambda ivec, uvec: DenseVector(np.array(ivec) * np.array(uvec)), VectorUDT())(dft.ivec, dft.uvec))
dft1

In [ ]:
dft2 = dft1.withColumn("mcount", dft1.icount * dft1.ucount)
dft2

In [ ]:
vecAssembler = VectorAssembler(inputCols=['mvec', 'mcount'], outputCol="features")

In [ ]:
dft3 = vecAssembler.transform(dft2)
dft3

In [ ]:
from pyspark.ml.classification import GBTClassifier

In [ ]:
gbtc = GBTClassifier(labelCol="purchase", maxIter=10)
gbtc_model = gbtc.fit(dft3)

In [ ]:
def transform(df):
    dft = df.join(dfri, on='item_id')\
    .join(dfru, on='user_id')\
    .coalesce(6);
    dft1 = dft.withColumn('mvec', F.udf(lambda ivec, uvec: DenseVector(np.array(ivec) * np.array(uvec)), VectorUDT())(dft.ivec, dft.uvec))
    dft2 = dft1.withColumn("mcount", dft1.icount * dft1.ucount)
    dft3 = vecAssembler.transform(dft2)
    return dft3

In [ ]:
# prediction
pred = gbtc.transform(transform(df_test)).cache()

In [ ]:
pred.show(5, True, True)

In [ ]:
pred.filter(pred.prediction > 0).count()

In [ ]:
row = pred.take(1)

In [ ]:
row

In [ ]:
pred_1 = pred.drop(pred.purchase)

In [ ]:
pred_1.printSchema()

In [ ]:
pred_2 = pred_1.withColumn('purchase', F.udf(lambda prob: float(prob[1]), FloatType())(pred_1.probability))

In [ ]:
res = pred_2.select(pred_2.user_id, pred_2.item_id, pred_2.purchase).orderBy(pred_2.user_id, pred_2.item_id)

In [ ]:
res.toPandas().to_csv('lab03.csv')

In [ ]:
!head -n 5 lab03.csv

# Пробуем намутить предсказания через ALS

## ALS Шаг 1: Схлопываем дубликаты по именам

In [17]:
df_train.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- purchase: integer (nullable = true)



In [18]:
@F.pandas_udf(IntegerType())
def replaceDuplicate(item_id_sr):
    return item_id_sr.apply(lambda item_id: cor_ids[item_id] if item_id in cor_ids else item_id)

In [19]:
df_train_1 = df_train.withColumn('item_id_nd', replaceDuplicate(df_train.item_id))

In [20]:
df_test_1 = df_test.withColumn('item_id_nd', replaceDuplicate(df_test.item_id))

# ALS Шаг 2: Делаем рейтинг из факта покупки

* 0 (не купил) - рейтинг 1
* 1 (купил) - рейтинг 10

In [24]:
df_train_2 = df_train_1.withColumn('rating', df_train_1.purchase * F.lit(9) + F.lit(1))

Можно сделать не константный рейтинг, а на основе частоты покупаемости итемов и покупательской способности юзера

In [21]:
from pyspark.sql.window import Window

In [50]:
window_item_id = Window.partitionBy("item_id_nd")
window_user_id = Window.partitionBy("user_id")

In [51]:
df_train_r = df_train_1.withColumn('r', 
                                   F.sum(df_train_1.purchase).over(window_item_id)
                                   / F.count(F.lit(1)).over(window_item_id)
                                   * F.sum(df_train_1.purchase).over(window_user_id)
                                   / F.count(F.lit(1)).over(window_user_id))

In [52]:
window_r = Window.orderBy('r')

In [53]:
df_train_nr = df_train_r.withColumn('nr', F.percent_rank().over(window_r))

Вместо percent_rank можно еще просто нормализацию сделать, посмотрим что тогда лучше себя покажет

In [60]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

In [63]:
from pyspark.ml import Pipeline

In [68]:
assembler_r = VectorAssembler(inputCols=['r'],outputCol='r_arr')

In [69]:
scaler_r = MinMaxScaler(inputCol='r_arr', outputCol='scaled_r')

In [72]:
pipeline_r = Pipeline(stages=[assembler_r, scaler_r])

In [74]:
df_train_nr = pipeline_r.fit(df_train_r).transform(df_train_r).withColumn('nr', F.udf(lambda v: float(v[0]),FloatType())('scaled_r'))

In [75]:
df_train_2 = df_train_nr.withColumn('rating', F.lit(5.5) + F.lit(4.5) * (df_train_nr.purchase * df_train_nr.nr - (F.lit(1) - df_train_nr.purchase) * (F.lit(1) - df_train_nr.nr)))

In [ ]:
df_train_2.orderBy('rating', ascending=False).show()

# ALS Обучаем модель

In [76]:
from pyspark.ml.recommendation import ALS
als = ALS(
         userCol="user_id", 
         itemCol="item_id_nd",
         ratingCol="rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop"
)

In [77]:
als_model = als.fit(df_train_2)

# ALS Делаем предсказания

In [78]:
predictions = als_model.transform(df_test_1)

In [ ]:
predictions.summary().show()

# ALS Нормализуем предсказания

In [79]:
assembler = VectorAssembler(inputCols=['prediction'],outputCol='prediction_arr')

In [80]:
scaler = MinMaxScaler(inputCol='prediction_arr', outputCol='scaled_prediction')

In [81]:
pipeline = Pipeline(stages=[assembler, scaler])

In [82]:
preds = pipeline.fit(predictions).transform(predictions)

In [83]:
result = preds.withColumn('purchase', F.udf(lambda v: float(v[0]),FloatType())(preds.scaled_prediction))\
    .select('user_id', 'item_id', 'purchase')\
    .orderBy('user_id', 'item_id')

In [84]:
result.toPandas().to_csv('lab03.csv')

In [85]:
spark.stop()

In [ ]:
import pickle
with open('gbtcmodel.pk', "wb") as f:
            pickle.dump(gbtc, f)

In [8]:
!hdfs dfs -copyToLocal lab05.csv/part-00000-98cefec9-508c-40f4-b366-3df2f52be2e1-c000.csv lab05.csv

In [7]:
!hdfs dfs -ls lab05.csv

Found 2 items
-rw-r--r--   3 dmitriy.kamaev dmitriy.kamaev          0 2022-11-02 15:36 lab05.csv/_SUCCESS
-rw-r--r--   3 dmitriy.kamaev dmitriy.kamaev    1190768 2022-11-02 15:36 lab05.csv/part-00000-98cefec9-508c-40f4-b366-3df2f52be2e1-c000.csv
